# Panorama da desigualdade de gênero no mercado de Inteligência Artificial no Brasil

## Prof. Dr. Neylson Crepalde

In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd

In [ ]:
sud = pd.read_csv("../RAIS_VINC_PUB_MG_ES_RJ/RAIS_VINC_PUB_MG_ES_RJ.txt", sep=';', encoding='latin1')
sud.head()

sud.columns

sud["CNAE 2.0 Classe"].dtype

sud = sud.loc[(sud["CNAE 2.0 Classe"] >= 62000) & (sud["CNAE 2.0 Classe"] < 62050) ,:]
sud.shape

sud.to_csv("../data/rais_mg_es_rj.csv", sep="\t", index=False)

del(sud)

In [ ]:
sp = dd.read_csv("../RAIS_VINC_PUB_SP/RAIS_VINC_PUB_SP.txt", sep=';', encoding="latin1", dtype={'CBO Ocupação 2002': 'object'})
sp = sp.loc[(sp["CNAE 2.0 Classe"] >= 62000) & (sp["CNAE 2.0 Classe"] < 62050) ,:]
print(sp.shape)
print(sp.columns)

computed_sp = sp.compute()

computed_sp.shape

computed_sp.to_csv("../data/rais_sp.csv", sep="\t", index=False)

del(computed_sp)

In [94]:
# Dar um merge de linhas nos dois bancos...
sul = pd.concat([
    pd.read_csv("../data/rais_mg_es_rj.csv", sep='\t', decimal=',', na_values=["{ñ class}", "{ñclass}"]),
    pd.read_csv("../data/rais_sp.csv", sep='\t', decimal=',', na_values=["{ñ class}", "{ñclass}"])
], axis=0)

In [95]:
# Filtra apenas salário mensal
sul = sul.loc[sul["Tipo Salário"] == 1, :]

In [96]:
# Filçtra apenas trabalhos não parciais
sul = sul.loc[sul["Ind Trab Parcial"] == 0, :]

In [97]:
# Corrige grau de escolaridade para obter valor 0
sul['Escolaridade após 2005'] -= 1

In [98]:
print(sul['Escolaridade após 2005'].unique())
sul['Escolaridade após 2005'].value_counts()

[ 0  1  3  4  5  6  7  8  9 10  2]


8     177215
6      40281
7      24173
9       2967
5       2409
4       1522
10       444
3        393
2        202
1        201
0         13
Name: Escolaridade após 2005, dtype: int64

In [99]:
# Coloca label em sexo
sul.loc[sul['Sexo Trabalhador'] == 1, 'Sexo Trabalhador'] = "M"
sul.loc[sul['Sexo Trabalhador'] == 2, 'Sexo Trabalhador'] = "F"

In [100]:
sul['Sexo Trabalhador'].value_counts()

M    166632
F     83188
Name: Sexo Trabalhador, dtype: int64

In [101]:
# Filtra apenas quem Raça Cor não é 99
sul = sul.loc[sul["Raça Cor"] != 99,:]

In [103]:
# Coloca label em raça
sul.loc[sul["Raça Cor"] == 1, 'Raça Cor'] = "Indígena"
sul.loc[sul["Raça Cor"] == 2, 'Raça Cor'] = "Branca"
sul.loc[sul["Raça Cor"] == 4, 'Raça Cor'] = "Preta"
sul.loc[sul["Raça Cor"] == 6, 'Raça Cor'] = "Amarela"
sul.loc[sul["Raça Cor"] == 8, 'Raça Cor'] = "Parda"
sul.loc[sul["Raça Cor"] == 9, 'Raça Cor'] = "Não identificada"

In [104]:
sul['Raça Cor'].value_counts()

Branca              159556
Parda                54080
Não identificada     21159
Preta                10504
Amarela               3829
Indígena               687
Name: Raça Cor, dtype: int64

In [122]:
sul['Brasileiro'] = sul['Nacionalidade'].apply(lambda x: 1 if (x == 10) else 0)

In [126]:
sul.Brasileiro.value_counts()

1    247948
0      1867
Name: Brasileiro, dtype: int64

Muito pouco estrangeiro trabalhando na área. Talvez não seja uma variável interessante para modelar.

In [129]:
sul.groupby("Brasileiro").agg({"Vl Remun Média Nom": "median"})

,Vl Remun Média Nom
Brasileiro,
0,6977.45
1,3571.28


Mas a diferença salarial é gritante! Investiguemos.

In [105]:
sul['Tempo Emprego']

0           1.0
1           2.9
2           7.7
3           2.9
4           2.9
          ...  
180452     35.9
180453    377.3
180454    378.9
180455    147.9
180456      6.4
Name: Tempo Emprego, Length: 249815, dtype: float64

In [106]:
sul.shape

(249815, 62)

In [107]:
sul.columns

Index(['Bairros SP', 'Bairros Fortaleza', 'Bairros RJ', 'Causa Afastamento 1',
       'Causa Afastamento 2', 'Causa Afastamento 3', 'Motivo Desligamento',
       'CBO Ocupação 2002', 'CNAE 2.0 Classe', 'CNAE 95 Classe',
       'Distritos SP', 'Vínculo Ativo 31/12', 'Faixa Etária',
       'Faixa Hora Contrat', 'Faixa Remun Dezem (SM)',
       'Faixa Remun Média (SM)', 'Faixa Tempo Emprego',
       'Escolaridade após 2005', 'Qtd Hora Contr', 'Idade',
       'Ind CEI Vinculado', 'Ind Simples', 'Mês Admissão', 'Mês Desligamento',
       'Mun Trab', 'Município', 'Nacionalidade', 'Natureza Jurídica',
       'Ind Portador Defic', 'Qtd Dias Afastamento', 'Raça Cor',
       'Regiões Adm DF', 'Vl Remun Dezembro Nom', 'Vl Remun Dezembro (SM)',
       'Vl Remun Média Nom', 'Vl Remun Média (SM)', 'CNAE 2.0 Subclasse',
       'Sexo Trabalhador', 'Tamanho Estabelecimento', 'Tempo Emprego',
       'Tipo Admissão', 'Tipo Estab', 'Tipo Estab.1', 'Tipo Defic',
       'Tipo Vínculo', 'IBGE Subsetor', 'Vl 

#### Variáveis de interesse para trabalhar

- Sexo Trabalhador
- Idade
- Qtd Hora Contrat
- Escolaridade após 2005
- Município
- Nacionalidade
- Ind Portador Defic
- Raça Cor
- Vl Remun Dezembro Nom
- Vl Remun Média Nom
- Tamanho Estabelecimento
- Tempo Emprego (em meses)
- Ind Portador Defic
- Brasileiro

Verificar as categorias das variáveis e implementar cosntrução de labels.

---

# Análise Exploratória de Dados

## Sexo

In [108]:
print(sul['Sexo Trabalhador'].value_counts())
print(((sul['Sexo Trabalhador'].value_counts() / sul.shape[0] )*100).round(2))

M    166628
F     83187
Name: Sexo Trabalhador, dtype: int64
M    66.7
F    33.3
Name: Sexo Trabalhador, dtype: float64


In [109]:
pd.crosstab(sul["Sexo Trabalhador"], sul["Faixa Hora Contrat"])

Faixa Hora Contrat,1,2,3,4,5,6
Sexo Trabalhador,,,,,,
F,1534,35,519,2325,62895,15879
M,2188,29,548,1973,132414,29476


In [110]:
pd.crosstab(sul["Sexo Trabalhador"], sul["Faixa Remun Dezem (SM)"])

Faixa Remun Dezem (SM),0,1,2,3,4,5,6,7,8,9,10,11,12,99
Sexo Trabalhador,,,,,,,,,,,,,,
F,25271,200,1323,6587,7850,9958,6610,4918,6585,5572,3951,1680,1638,1044
M,50152,236,1037,4858,11503,17746,12603,9706,15311,16612,13423,5683,6137,1621


In [111]:
pd.crosstab(sul["Sexo Trabalhador"], sul["Faixa Remun Média (SM)"])

Faixa Remun Média (SM),0,1,2,3,4,5,6,7,8,9,10,11,99
Sexo Trabalhador,,,,,,,,,,,,,
F,294,2177,13093,12369,14061,9221,6551,8784,7093,4603,1915,1830,1196
M,339,1697,10646,19688,26462,17896,13788,21759,22317,16638,6585,6942,1871


In [112]:
pd.crosstab(sul["Sexo Trabalhador"], sul["Faixa Tempo Emprego"])

Faixa Tempo Emprego,1,2,3,4,5,6,7,8,99
Sexo Trabalhador,,,,,,,,,
F,11680,8087,13699,15400,8770,10838,9957,4665,91
M,20772,17873,27843,31188,17145,22544,21259,7842,162


In [113]:
sul.groupby("Sexo Trabalhador").agg({"Idade": "mean"})

,Idade
Sexo Trabalhador,
F,33.558837
M,34.006998


In [114]:
sul.groupby("Sexo Trabalhador").agg({"Vl Remun Média Nom": "median"})

,Vl Remun Média Nom
Sexo Trabalhador,
F,2727.070
M,4128.185


In [115]:
sul.groupby("Sexo Trabalhador").agg({"Vl Remun Média Nom": "mean"})

,Vl Remun Média Nom
Sexo Trabalhador,
F,4489.264065
M,6233.341422
